# Estimate car price - First Exploratory Data Analysis and Imputation
#### Andreas Forster

### Introduction
-  Push calculations down to SAP HANA. 
-  Leverage the in-memory performance directly out of Python without data extraction.

### Documentation
-  SAP HANA Python Client API for Machine Learning Algorithms:   
   https://help.sap.com/doc/0172e3957b5946da85d3fde85ee8f33d/latest/en-US/html/hana_ml.html
-  SAP HANA Predictive Analysis Library (PAL):  
   https://help.sap.com/viewer/2cfbc5cf2bc14f028cfbe2a2bba60a50/latest/en-US/f652a8186a144e929a1ade7a3cb7abe8.html
-  Dataset: https://www.kaggle.com/bozungu/ebay-used-car-sales-data

### Create a HANA DataFrame, which points to the training data that remains in SAP HANA

In [1]:
# Obtain logon details from the central Connection Management of SAP Data Intelligence
import hana_ml.dataframe as dataframe

# Instantiate connection object
from notebook_hana_connector.notebook_hana_connector import NotebookConnectionContext
conn = NotebookConnectionContext(connectionId='HANADB')

# Create the HANA dataframe in the structure of the specified table / view
df_pushdown = conn.table("USEDCARPRICES", schema="TAXX")

### Peek at the data and retrieve a small number of rows

In [2]:
df_pushdown.head(5).collect()

,CAR_ID,VEHICLETYPE,NAME,MODEL,YEAR,GEARBOX,HP,FUELTYPE,KILOMETER,DAMAGED,PRICE
0,16.0,limousine,Mercedes_C_200_Kompressor,C-Class,2000.0,manual,160.0,petrol,150000.0,no,2300.0
1,69.0,limousine,C_180_mit_TÜV,C-Class,1995.0,manual,122.0,petrol,150000.0,no,600.0
2,82.0,limousine,Mercedes_Benz_C_230,C-Class,1996.0,None,150.0,petrol,150000.0,no,800.0
3,97.0,limousine,Mercedes_C200_AMG_Optik,C-Class,2002.0,manual,180.0,petrol,150000.0,no,2850.0
4,111.0,limousine,Mercedes_Benz_C_180,C-Class,2001.0,automatic,129.0,petrol,150000.0,no,2400.0


In [3]:
df_pushdown.agg([("max", "HP", "MAXPOWER")]).collect()

,MAXPOWER
0,20000.0


### Get row count

In [4]:
df_pushdown.count()

26900

### Get data types

In [5]:
df_pushdown.dtypes()

[('CAR_ID', 'DOUBLE', 15),
 ('VEHICLETYPE', 'NVARCHAR', 200),
 ('NAME', 'NVARCHAR', 200),
 ('MODEL', 'NVARCHAR', 200),
 ('YEAR', 'DOUBLE', 15),
 ('GEARBOX', 'NVARCHAR', 200),
 ('HP', 'DOUBLE', 15),
 ('FUELTYPE', 'NVARCHAR', 200),
 ('KILOMETER', 'DOUBLE', 15),
 ('DAMAGED', 'NVARCHAR', 200),
 ('PRICE', 'DOUBLE', 15)]

### Get column statistics

In [6]:
df_pushdown.describe().collect()

,column,count,unique,nulls,mean,std,min,max,median,25_percent_cont,25_percent_disc,50_percent_cont,50_percent_disc,75_percent_cont,75_percent_disc
0,CAR_ID,26900,26900,0,13449.500000,7765.505457,0.0,26899.0,13449.5,6724.75,6724.0,13449.5,13449.0,20174.25,20174.0
1,YEAR,26900,74,0,2003.002342,31.352877,1000.0,6200.0,2003.0,1999.00,1999.0,2003.0,2003.0,2008.00,2008.0
2,HP,26900,307,0,163.146171,212.759226,0.0,20000.0,150.0,122.00,122.0,150.0,150.0,193.00,193.0
3,KILOMETER,26900,13,0,129136.059480,36850.310066,5000.0,150000.0,150000.0,125000.00,125000.0,150000.0,150000.0,150000.00,150000.0
4,PRICE,26900,1972,0,10078.361970,86154.852061,0.0,14000500.0,6000.0,2800.00,2800.0,6000.0,6000.0,12887.25,12887.0
5,VEHICLETYPE,25899,8,1001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NAME,26900,15470,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,MODEL,26381,18,519,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,GEARBOX,26396,2,504,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,FUELTYPE,25930,7,970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Display the underlying SELECT statement for transparency / verification / modification
#df_pushdown.describe().select_statement

### Display some of the rows with missing data

In [8]:
df_pushdown.filter("GEARBOX IS NULL").head(10).collect()

,CAR_ID,VEHICLETYPE,NAME,MODEL,YEAR,GEARBOX,HP,FUELTYPE,KILOMETER,DAMAGED,PRICE
0,82.0,limousine,Mercedes_Benz_C_230,C-Class,1996.0,None,150.0,petrol,150000.0,no,800.0
1,2412.0,limousine,Top_C_klas?_voll_austatung,C-Class,2000.0,None,163.0,petrol,150000.0,no,2999.0
2,5030.0,limousine,Mercedes_Benz_C_180,C-Class,1995.0,None,0.0,petrol,150000.0,no,1750.0
3,5797.0,limousine,C_180_Elegance_/_Motor_und_Getriebe_1a_/_Tuev_Neu,C-Class,1995.0,None,0.0,petrol,150000.0,no,1150.0
4,7217.0,limousine,Mercedes_benz_C200_Kompressor_tuev_neu,C-Class,2001.0,None,163.0,petrol,150000.0,no,3700.0
5,12767.0,limousine,Mercedes_Benz_W202_C200_Elegance,C-Class,1996.0,None,136.0,petrol,150000.0,no,2000.0
6,15406.0,limousine,Mercedes_Benz_Avantgarde_AMG_Navi_Comand_SB_SH...,C-Class,2001.0,None,163.0,petrol,150000.0,no,4999.0
7,18276.0,limousine,Mercedes_C180_Esprit_Tuev_02/2017,C-Class,1998.0,None,122.0,petrol,150000.0,no,1450.0
8,4878.0,None,C_Kombi_200,C-Class,2016.0,None,0.0,petrol,150000.0,no,990.0
9,6364.0,None,MERCEDES_KOMBI/180T/122PS/AUTOMATIC/w210,C-Class,2018.0,None,0.0,petrol,150000.0,no,1950.0


### Impute unknown GEARBOX in new column
View row in which the value was imputed

In [9]:
df_pushdown = df_pushdown.select(df_pushdown.columns, ('CASE WHEN "GEARBOX" IS NULL THEN \'gearbox unknown\' ELSE "GEARBOX" END', 'GEARBOX_IMP'))
df_pushdown.filter("CAR_ID = 3160").collect()

,CAR_ID,VEHICLETYPE,NAME,MODEL,YEAR,GEARBOX,HP,FUELTYPE,KILOMETER,DAMAGED,PRICE,GEARBOX_IMP
0,3160.0,kombi,Mercedes_Benz_E_270_T_CDI_Avantgarde_Voll_Aust...,E-Class,2003.0,automatic,177.0,diesel,150000.0,no,5400.0,automatic


### Drop the original GEARBOX column with missing values

In [10]:
### Drop the original column with missing values
df_pushdown = df_pushdown.drop(["GEARBOX"])

In [11]:
# Move the imputed column to the front, which keeps the target table PRICE at the end
df_pushdown = df_pushdown.to_head("GEARBOX_IMP")

In [12]:
df_pushdown.describe().collect()

,column,count,unique,nulls,mean,std,min,max,median,25_percent_cont,25_percent_disc,50_percent_cont,50_percent_disc,75_percent_cont,75_percent_disc
0,CAR_ID,26900,26900,0,13449.500000,7765.505457,0.0,26899.0,13449.5,6724.75,6724.0,13449.5,13449.0,20174.25,20174.0
1,YEAR,26900,74,0,2003.002342,31.352877,1000.0,6200.0,2003.0,1999.00,1999.0,2003.0,2003.0,2008.00,2008.0
2,HP,26900,307,0,163.146171,212.759226,0.0,20000.0,150.0,122.00,122.0,150.0,150.0,193.00,193.0
3,KILOMETER,26900,13,0,129136.059480,36850.310066,5000.0,150000.0,150000.0,125000.00,125000.0,150000.0,150000.0,150000.00,150000.0
4,PRICE,26900,1972,0,10078.361970,86154.852061,0.0,14000500.0,6000.0,2800.00,2800.0,6000.0,6000.0,12887.25,12887.0
5,GEARBOX_IMP,26900,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,VEHICLETYPE,25899,8,1001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NAME,26900,15470,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,MODEL,26381,18,519,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,FUELTYPE,25930,7,970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Impute missing continuous values with column average
This is an example on how a numerical column can be imputed. It is not required for this very project as there are no missing continuous values in the data. However, this code may come useful for your own projects and datasets.

In [13]:
# Get column average
avg = df_pushdown.agg([('avg', 'PRICE', 'PRICE_MEAN')]).collect()
col_avg = round(float(avg.iloc[0:,0]))
col_avg

10078

In [14]:
# Replace missing NAs with column average
df_pushdown = df_pushdown.fillna(col_avg, ["PRICE"])

### Drop remaining rows with missing values

In [15]:
df_pushdown = df_pushdown.dropna()

In [16]:
df_pushdown.count()

25019

### Change column type of the key column
This is required for some PAL algorithms, ie the Correlation Matrix

In [17]:
df_pushdown = df_pushdown.cast('CAR_ID', 'INTEGER')

### Save the imputed data as view (no physical table)

In [19]:
#df_pushdown.save(where="CARS_IMP", table_type="COLUMN")
df_pushdown.save(where=("TAXX", "USEDCARPRICES_IMP_VIEW"), table_type="VIEW")

### Close connection

In [21]:
conn.close()